In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
folder_path = [FOLDER_PATH]

Mounted at /content/drive


ベースラインとして、train_dataの11月と12月データだけ考えてみる(気温の影響は前月の値段に一定含有されているので一旦無視)


具体的には2019年11月のデータに、2016と2018の12月/11月比を用いて補正する。(2017は動態が微妙に異なるため除外、excelの簡易計算でも2016.18の結果を使う方が良い値)

In [2]:
train_df = pd.read_csv(folder_path + "train_data.csv")
train_df.set_index('id', inplace=True)
train_df = train_df.T

In [3]:
nov_dec_ratio_2016 = train_df["2016-12-01"] / train_df["2016-11-01"]
nov_dec_ratio_2018 = train_df["2018-12-01"] / train_df["2018-11-01"]
nov_dec_ratio = (nov_dec_ratio_2016 + nov_dec_ratio_2018) / 2
pred = (train_df["2019-11-01"] * nov_dec_ratio).round(0)# （参考：ここまででpublic 0.14525、private 0.16944

一通り価格推移を眺めてみて、特異な３種類の野菜だけ特別な対応をいれる

In [4]:
#みつばは12月に必ず高騰する。2016-18の12月の平均値を使うことにする
dec_ave = ((train_df["2016-12-01"] + train_df["2017-12-01"] + train_df["2018-12-01"]) / 3).round(0)
combined_df = pd.concat([pred, dec_ave], axis=1, keys=['pred', 'pred_sub'])
condition = combined_df.index.str.contains('みつば')
pred = pd.Series([combined_df.loc[idx, 'pred_sub'] if cond else combined_df.loc[idx, 'pred'] for idx, cond in zip(combined_df.index, condition)], index=combined_df.index)

In [5]:
#キャベツは基本的に上がったり下がったりをする。12月/11月比がx0.6-0.7になってしまっているが、これはたまたま。2019/11の値のままにする
combined_df = pd.concat([train_df["2019-11-01"], pred], axis=1, keys=['nov_val', 'pred'])
condition = combined_df.index.str.contains('キャベツ')
pred = pd.Series([combined_df.loc[idx, 'nov_val'] if cond else combined_df.loc[idx, 'pred'] for idx, cond in zip(combined_df.index, condition)], index=combined_df.index)

In [6]:
#にんじんは直近安定しているが過去の暴騰暴落の影響で12月/11月比がx0.67されている。小さい価格なので影響が大きい。最後は横ばいなので2019/11の値のままにする
combined_df = pd.concat([train_df["2019-11-01"], pred], axis=1, keys=['nov_val', 'pred'])
condition = combined_df.index.str.contains('にんじん')
pred = pd.Series([combined_df.loc[idx, 'nov_val'] if cond else combined_df.loc[idx, 'pred'] for idx, cond in zip(combined_df.index, condition)], index=combined_df.index)

過去の5%-95%の値に収める(評価指標的に、大外れすると悪影響が大きいので)

In [ ]:
min_df = train_df.min(axis=1)
max_df = train_df.max(axis=1)
diff = max_df - min_df

min_df = min_df + diff * 0.05
max_df = max_df - diff * 0.05
pred = np.where(pred < min_df, min_df, pred)
pred = np.where(pred > max_df, max_df, pred)
pred = pd.Series(pred,index=train_df.index).round(0)

In [ ]:
#リリース処理
pred_upd = pd.DataFrame(pred).reset_index()
pred_upd.columns = ["id", "y"]
pred_upd.to_csv("submit.csv", index=False) #public 0.12259、private 0.14407